In [23]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score


np.random.seed(500)

Corpus = pd.read_csv("resource_catalog_with_FOE.csv", dtype=object)
Corpus.head()

,uni_id,course_name,title,authors,publication_year,resource_type,field_of_education
0,3,swallowing across the lifespan,pathology of the head and neck,antoniocardesa,2006.0,book,health
1,3,swallowing across the lifespan,bacterial pathogenesis a subject collection fr...,pascalecossart,2014.0,book,health
2,3,swallowing across the lifespan,the use of the laryngoscope in diseases of the...,morellmackenzie,1865.0,book,health
3,3,swallowing across the lifespan,oxford american handbook of physical medicine ...,lynweiss,2010.0,book,health
4,3,borderlands identity culture and belonging,diogenes,markusher,2009.0,book,society


In [24]:
# Step - a : Remove blank rows if any.
Corpus['title'].dropna(inplace=True)
# Step - b : Change all the text to lower case. This is required as python interprets 'dog' and 'DOG' differently
Corpus['title'] = [entry.lower() for entry in Corpus['title']]
# Step - c : Tokenization : In this each entry in the corpus will be broken into set of words
Corpus['title']= [word_tokenize(entry) for entry in Corpus['title']]
# Step - d : Remove Stop words, Non-Numeric and perfom Word Stemming/Lemmantzing.
# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
for index,entry in enumerate(Corpus['title']):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    Corpus.loc[index,'text_final'] = str(Final_words)

In [25]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(Corpus['text_final'],Corpus['field_of_education'],test_size=0.20)

In [26]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

In [27]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(Corpus['text_final'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [28]:
print(Tfidf_vect.vocabulary_)

{'pathology': 3252, 'head': 1993, 'neck': 2973, 'bacterial': 367, 'pathogenesis': 3250, 'subject': 4337, 'collection': 774, 'cold': 769, 'spring': 4253, 'perspective': 3314, 'medicine': 2749, 'use': 4755, 'disease': 1250, 'appendix': 221, 'oxford': 3185, 'american': 159, 'handbook': 1970, 'physical': 3355, 'rehabilitation': 3783, 'post': 3450, 'colonial': 780, 'study': 4329, 'key': 2430, 'concept': 853, 'feminist': 1663, 'cultural': 1041, 'transnational': 4622, 'tourism': 4569, 'experience': 1577, 'gallipoli': 1808, 'organisational': 3145, 'behaviour': 418, 'core': 965, 'application': 225, 'gm': 1887, 'change': 661, 'management': 2660, 'creativity': 1007, 'theory': 4510, 'history': 2041, 'practice': 3467, 'paediatrics': 3190, 'present': 3499, 'future': 1804, 'sport': 4249, 'education': 1361, 'asia': 279, 'pacific': 3186, 'region': 3770, 'proceed': 3540, 'congress': 879, 'university': 4724, 'president': 3503, 'june': 2404, 'surgery': 4384, 'surgeon': 4383, 'edinburgh': 1354, 'sydney': 4

In [29]:
print(Train_X_Tfidf)

  (0, 4418)	0.345867959499327
  (0, 3908)	0.2676013890778758
  (0, 3030)	0.4449998679350105
  (0, 2006)	0.4348015745072705
  (0, 1800)	0.36921413988810403
  (0, 1593)	0.373788237914311
  (0, 166)	0.3816422822523558
  (1, 4951)	0.30789410058768063
  (1, 4378)	0.29899143374136583
  (1, 3541)	0.2397599952188917
  (1, 2660)	0.17411139086560218
  (1, 2189)	0.26786115773088987
  (1, 2045)	0.30361428747648594
  (1, 1180)	0.25396823079940367
  (1, 991)	0.27038393552713563
  (1, 652)	0.3052746610652895
  (1, 373)	0.32690000405358227
  (1, 325)	0.3044363346630388
  (1, 113)	0.3656903474347313
  (2, 3059)	0.3519537339799906
  (2, 2887)	0.38142001349382765
  (2, 2299)	0.48600621707287694
  (2, 1958)	0.5163314820099137
  (2, 1157)	0.4773333217950201
  (3, 3733)	0.713963815678234
  :	:
  (19922, 2503)	0.3042859385818741
  (19922, 1970)	0.2721796568476339
  (19922, 1236)	0.398670983980515
  (19922, 766)	0.39632108376855824
  (19922, 283)	0.43377195431863314
  (19923, 3833)	0.7313424531299657
  (19923

In [30]:
# fit the training dataset on the NB classifier
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_NB = Naive.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score: ",accuracy_score(predictions_NB, Test_Y)*100)

Naive Bayes Accuracy Score:  69.57656030503713
